In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep = '\t')

In [22]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [23]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X = df['message']

In [26]:
y = df['label']

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

## Count Vector

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
count_vect = CountVectorizer()

## Fit the Vectorizer to the data

In [30]:
count_vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

## Transform the original text message

In [32]:
X_train_counts = count_vect.transform(X_train)

In [33]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [34]:
X_train_counts.shape

(3733, 7082)

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer

In [37]:
tfid_tranformer = TfidfTransformer()

In [38]:
X_train_tfid = tfid_tranformer.fit_transform(X_train_counts)

In [40]:
X_train_tfid.shape

(3733, 7082)

## Perform CountVectorizer and TfidfTransformer in single step

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer()

In [43]:
X_train_tfidf = vectorizer.fit_transform(X_train)

In [45]:
from sklearn.svm import LinearSVC

In [46]:
svc_cls = LinearSVC()

In [47]:
svc_cls.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

## Pipeline

In [51]:
from sklearn.pipeline import Pipeline

In [52]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])

In [54]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [57]:
predictions  = text_clf.predict(X_test)

In [59]:
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [65]:
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score

In [66]:
print(confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [67]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [68]:
print(accuracy_score(y_test,predictions))

0.989668297988037


In [82]:
text_clf.predict(['You have won a free $10 appointment'])

array(['spam'], dtype=object)